<a href="https://colab.research.google.com/github/arthurnicolas59/BloodCellDec22---DataScientest/blob/Ludo/3_Segmentation_de_4500_images_Modele_SAM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Connexion au Drive, activation du GPU, import des images**


### **Activation du GPU**



In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

Default GPU Device:/device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


### **Import des images (Barcelone)**



In [ ]:
!unzip 'drive/MyDrive/Datascientest/PBC_dataset_normal_DIB.zip'

In [ ]:
# suppression d'une image présentant un défaut de format :
import os

os.remove('/content/PBC_dataset_normal_DIB/neutrophil/.DS_169665.jpg')

# **2. Création du dataframe 'df' identifiant le chemin d'accès de chaque image et son label**

### **Dataframe**

In [ ]:
import glob
import pandas as pd

# Trouver tous les chemins vers les fichiers qui finissent par .jpg
liste = glob.glob('./PBC_dataset_normal_DIB/*/*.jpg')

# Création d'une liste comprenant les chemins d'accès aux images ainsi que le label de chaque image
liste = list(map(lambda x : [x, x.split('/')[3].split('_')[0]], liste))

# Créer un DataFrame pandas
df = pd.DataFrame(liste, columns=['filepath', 'nameLabel'])
df=df.replace(['SNE','BNE'],'NEUTROPHIL')
df=df.replace(['MY','MMY','PMY'],'IG')
df['label'] = df['nameLabel'].replace(df.nameLabel.unique(), [*range(len(df.nameLabel.unique()))])
df.sort_values('filepath')
df

,filepath,nameLabel,label
0,./PBC_dataset_normal_DIB/basophil/BA_463171.jpg,BA,0
1,./PBC_dataset_normal_DIB/basophil/BA_307830.jpg,BA,0
2,./PBC_dataset_normal_DIB/basophil/BA_317405.jpg,BA,0
3,./PBC_dataset_normal_DIB/basophil/BA_4118.jpg,BA,0
4,./PBC_dataset_normal_DIB/basophil/BA_310106.jpg,BA,0
...,...,...,...
17087,./PBC_dataset_normal_DIB/platelet/PLATELET_220...,PLATELET,7
17088,./PBC_dataset_normal_DIB/platelet/PLATELET_235...,PLATELET,7
17089,./PBC_dataset_normal_DIB/platelet/PLATELET_234...,PLATELET,7
17090,./PBC_dataset_normal_DIB/platelet/PLATELET_158...,PLATELET,7


### **Information sur le dataset**

In [ ]:
# shape
df.nameLabel.unique()
df.shape

(17092, 3)

In [ ]:
# répartition du nombre d'images selon le label
df.nameLabel.value_counts()

NEUTROPHIL    3329
EO            3117
IG            2895
PLATELET      2348
ERB           1551
MO            1420
BA            1218
LY            1214
Name: nameLabel, dtype: int64

# **3. Création d'images SEGMENTEES à l'aide du modèle The Segment Anything Model (SAM)**

Méthode décrite dans ce notebook :
https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb


### **1. Configuration de l'environnement d'exécution**

In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-0rgg6ass
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-0rgg6ass
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=f7d9d6081b4e8e7caebc217fbc48e103347a3bedb342777d35929d5e543ea672
  Stored in directory: /tmp/pip-ephem-wheel-cache-vwz5mtt9/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2023-07-19 11:55:19--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.15, 52.84.251.114, 5

In [ ]:
# chargement de la librairie supervision pour la visualisation
!pip install supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
### Import des modules nécessaires pour l'ensemble du travail
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import supervision as sv
from skimage import io
from skimage import filters
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor
from google.colab import drive
import os
from tqdm import tqdm
from PIL import Image
import imageio
import time

### **2. Création des fonctions permettant de visualiser la zone d'intérêt de l'image pour la segmentation**

In [ ]:
# affichage de la boxe délimitant la zone de recherche

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

# affichage d'un point d'intérêt dans la boxe délimitant la zone de recherche

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

# affichage du masque

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

### **3. Chargement du modèle et du prédicteur**

In [ ]:
# chargement du modèle
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

### **4. Application du modèle sur le dataset d'images**

**Création d'un dossier local "masques" et des 8 sous-dossiers correspondant aux groupes de cellules**

In [ ]:
# Chemin du répertoire principal de Google Drive
main_dir_masques='/content/'

# Nom du sous-dossier à créer
subfolder_name = 'masques'

# Chemin complet du sous-dossier à créer
subfolder_path = os.path.join(main_dir_masques, subfolder_name)

# Vérifier si le sous-dossier existe déjà
if not os.path.exists(subfolder_path):
    # Créer le sous-dossier
    os.makedirs(subfolder_path)
    print("Le sous-dossier '{}' a été créé avec succès.".format(subfolder_name))
else:
    print("Le sous-dossier '{}' existe déjà.".format(subfolder_name))

main_dir='/content/PBC_dataset_normal_DIB'

liste_dossier=[]
for folder_idx, folder in enumerate(os.listdir(path=main_dir)):
  liste_dossier.append(folder)

print(liste_dossier)

for folder_name in liste_dossier:
  group_path=os.path.join(subfolder_path,folder_name)

  # Vérifier si le sous-dossier existe déjà
  if not os.path.exists(group_path):
    # Créer le sous-dossier
    os.makedirs(group_path)
    print("Le sous-dossier '{}' a été créé avec succès.".format(folder_name))
  else:
    print("Le sous-dossier '{}' existe déjà.".format(folder_name))

Le sous-dossier 'masques' existe déjà.
['monocyte', 'lymphocyte', 'neutrophil', 'basophil', 'erythroblast', 'platelet', 'ig', 'eosinophil']
Le sous-dossier 'monocyte' existe déjà.
Le sous-dossier 'lymphocyte' existe déjà.
Le sous-dossier 'neutrophil' existe déjà.
Le sous-dossier 'basophil' existe déjà.
Le sous-dossier 'erythroblast' existe déjà.
Le sous-dossier 'platelet' existe déjà.
Le sous-dossier 'ig' existe déjà.
Le sous-dossier 'eosinophil' existe déjà.


**Génération de masques par groupe de cellules et sauvegarde dans le répertoire local "masques":
Génération de masques par tranches de 100 images par label**

In [ ]:
# Définition des dimensions et positions des boxes et points appliqués sur chaque images pour limiter la segmentation a une partie restreinte de l'image
input_box = np.array([90, 90, 270, 270])
input_point = np.array([[180, 180]])
input_label = np.array([1]) # signifie que les pixels sélectionnés représentant le masque positif de l'image

# Copie de df
train_df=df

# Définition d'une liste contenant les différents labels
liste_label =df.nameLabel.unique()

# Parcours et écriture des masques de chaque groupe présent dans la liste "liste_label"
# Ici on prend uniquement une centaine d'images par groupe
for label in liste_label :
  MASK_DF = train_df.loc[train_df['nameLabel'] == label][400:500]
  IMAGE_PATH = np.array(MASK_DF['filepath'])

  # Création et enregistrement des masques pour chaque image

  for i in tqdm(range(len(MASK_DF))):

          # Read image
          image_bgr = cv2.imread(IMAGE_PATH[i])
          image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

          start_time = time.time()
          predictor.set_image(image_rgb)
          end_time = time.time()
          set_image_time = end_time - start_time
          print("Temps pris par predictor.set_image() :", set_image_time, "secondes")
          start_time = time.time()
          masque, _, _ = predictor.predict(
            point_coords=None,   # pas de ciblage de la zone d'intérêt avec un point
            point_labels=None,
            box=input_box,        # ciblage de la zone d'intérêt avec une box
            multimask_output=False,
            )

          # Chemin du dossier où vous souhaitez enregistrer l'image
          output_folder = '/content/masques/' + IMAGE_PATH[i].split('/')[2]


          # Nom du fichier de sortie
          output_filename = str(MASK_DF['filepath'].iloc[i]).split('/')[3].split('.')[0]+'_mask.png'

          # Chemin complet du fichier de sortie
          output_path = output_folder+'/'+output_filename

          mask_image = np.uint8(masque[0] * 255)

          # Save the mask image
          cv2.imwrite(output_path, mask_image)

          # Enregistrez l'image dans le fichier PNG
          print('L\'image n°', i, 'a été enregistrée sous :', output_path)

 50%|█████     | 1/2 [00:02<00:02,  2.05s/it]

Temps pris par predictor.set_image() : 1.9466114044189453 secondes
L'image n° 0 a été enregistrée sous : /content/masques/basophil/BA_785668_mask.png


100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


Temps pris par predictor.set_image() : 1.938373327255249 secondes
L'image n° 1 a été enregistrée sous : /content/masques/basophil/BA_598745_mask.png


 50%|█████     | 1/2 [00:02<00:02,  2.04s/it]

...

Temps pris par predictor.set_image() : 1.9885985851287842 secondes
L'image n° 1 a été enregistrée sous : /content/masques/basophil/BA_598745_mask.png


 50%|█████     | 1/2 [00:02<00:02,  2.06s/it]

Temps pris par predictor.set_image() : 1.9761207103729248 secondes
L'image n° 0 a été enregistrée sous : /content/masques/basophil/BA_785668_mask.png


100%|██████████| 2/2 [00:04<00:00,  2.06s/it]

Temps pris par predictor.set_image() : 1.9633045196533203 secondes
L'image n° 1 a été enregistrée sous : /content/masques/basophil/BA_598745_mask.png


# **4. Tri manuel des images segmentées dans les sous dossiers du dossier masques (suppression des segmentations ratées)**

# **5. Création du dataframe 'df_mask' présentant le chemin d'accès des images segmentées issues du modèle SAM et des images sources correspondantes**

### **Dataframe**

In [ ]:
# Trouver tous les chemins vers les fichiers qui finissent par .jpg
liste_mask = glob.glob('/content/masques/*/*.png')

# # Création d'une liste comprenant les chemins d'accès aux images ainsi que le label de chaque image
liste_mask = list(map(lambda x : [x,
                                  x.split('/')[4].split('_')[0],
                                  '/content/PBC_dataset_normal_DIB/'+x.split('/')[3]+'/'+ x.split('/')[4].split('_')[0]+'_'+x.split('/')[4].split('_')[1]+'.jpg'],
                       liste_mask))

# Créer un DataFrame pandas
df_mask = pd.DataFrame(liste_mask, columns=['maskpath', 'nameLabel','filepath'])
df_mask=df_mask.replace(['SNE','BNE'],'NEUTROPHIL')
df_mask=df_mask.replace(['MY','MMY','PMY'],'IG')
df_mask['label'] = df['nameLabel'].replace(df_mask.nameLabel.unique(), [*range(len(df_mask.nameLabel.unique()))])
df_mask.sort_values('maskpath')
df_mask

,maskpath,nameLabel,filepath,label
0,/content/masques/basophil/BA_462792_mask.png,BA,/content/PBC_dataset_normal_DIB/basophil/BA_46...,0
1,/content/masques/basophil/BA_85940_mask.png,BA,/content/PBC_dataset_normal_DIB/basophil/BA_85...,0
2,/content/masques/basophil/BA_245418_mask.png,BA,/content/PBC_dataset_normal_DIB/basophil/BA_24...,0
3,/content/masques/basophil/BA_736344_mask.png,BA,/content/PBC_dataset_normal_DIB/basophil/BA_73...,0
4,/content/masques/basophil/BA_271614_mask.png,BA,/content/PBC_dataset_normal_DIB/basophil/BA_27...,0
...,...,...,...,...
4497,/content/masques/platelet/PLATELET_188467_mask...,PLATELET,/content/PBC_dataset_normal_DIB/platelet/PLATE...,1
4498,/content/masques/platelet/PLATELET_938834_mask...,PLATELET,/content/PBC_dataset_normal_DIB/platelet/PLATE...,1
4499,/content/masques/platelet/PLATELET_962630_mask...,PLATELET,/content/PBC_dataset_normal_DIB/platelet/PLATE...,1
4500,/content/masques/platelet/PLATELET_220047_mask...,PLATELET,/content/PBC_dataset_normal_DIB/platelet/PLATE...,1


### **Information sur le dataset des 4502 images segmentées**

In [ ]:
# répartition du nombre d'images selon le label dans le dataframe df_mask
df_mask.nameLabel.value_counts()

BA            756
MO            721
LY            688
ERB           503
PLATELET      499
EO            455
NEUTROPHIL    443
IG            437
Name: nameLabel, dtype: int64